# Metryki w przestrzeni napisów

Zadanie dotyczy różnych metryk w przestrzeni napisów.

1. Zaimplementuj przynajmniej 3 "metryki" spośród wymienionych: cosinusowa, LCS, DICE, euklidesowa, Levenshteina.
2. Zaimplementuj przynajmniej 1 sposoby oceny jakości klasteryzacji (np. indeks Daviesa-Bouldina).
3. Stwórz stoplistę najczęściej występujących słów i zastosuj ją jako pre-processing dla nazw. Algorytmy klasteryzacji powinny działać na dwóch wariantach: z pre-processingiem i bez pre-processingu.
4. Wykonaj klasteryzację zawartości załączonego pliku (lines.txt) przy użyciu  metryk zaimplementowanych w pkt. 1. Każda linia to adres pocztowy firmy, różne sposoby zapisu tego samego adresu powinny się znaleźć w jednym klastrze.
5. Porównaj jakość wyników sposobami zaimplementowanymi w pkt. 2.
6. Czy masz jakiś pomysł na poprawę jakości klasteryzacji w tym zadaniu?

Sprawozdanie powinno zawierać porównanie wyników wszystkich metryk z użyciem stoplisty i bez.

Można jako wzorcową klasteryzację użyć pliku clusters.txt.



### Useful imports

In [123]:
import numpy as np
from sklearn.cluster import DBSCAN
from collections import Counter

### Ex 1

In [128]:
def lcs_metric(x, y):
    """
    Calculates the Longest Common Subsequence (LCS) metric between two strings.
    """
    if len(x) == 0 or len(y) == 0:
        return 1
    tab = [[None for j in range(len(y)+1)] for i in range(len(x)+1)]
    max_lcs = 0
    for i in range(len(x)+1):
        for j in range(len(y)+1):
            if i == 0 or j == 0:
                tab[i][j] = 0
            else:
                if x[i-1] == y[j-1]:
                    tab[i][j] = 1 + tab[i-1][j-1]
                else:
                    tab[i][j] = 0
            if tab[i][j] > max_lcs:
                max_lcs = tab[i][j]
    return 1 - max_lcs/max(len(x), len(y))

In [69]:
def make_ngram_vec(x, N=2):
    vec = {}
    for i in range(len(x)-N+1):
        if x[i:i+N] in vec:
            vec[x[i:i+N]] += 1
        else:
            vec[x[i:i+N]] = 1
    return vec

In [129]:
def cosine_metric(a, b):
    """
    Calculates the Cosine metric between two strings.
    """
    if len(a) == 0 or len(b) == 0:
        return 1
    a = set(make_ngram_vec(a).keys())
    b = set(make_ngram_vec(b).keys())
    return 1 - len(a.intersection(b)) / np.sqrt(len(a)*len(b))

In [76]:
def euclidean_metric(a, b):
    """
    Calculates the Euclidean metric between two strings.
    """
    a = make_ngram_vec(a)
    b = make_ngram_vec(b)
    keys = set(a.keys()).union(set(b.keys()))
    return np.sqrt(sum((a.get(k, 0) - b.get(k, 0))**2 for k in keys))

In [126]:
def dice_metric(a, b):
    """
    Calculates the Dice metric between two strings.
    """
    a = set(make_ngram_vec(a).keys())
    b = set(make_ngram_vec(b).keys())
    if len(a) == 0 or len(b) == 0:
        return 1

    return 1 - (2 * len(a.intersection(b))) / (len(a) + len(b))

### Ex 2

In [103]:
def find_centroid(cluster, metric):
    """
    Calculates the centroids of the clusters.
    """
    dists = [0 for i in cluster]
    for i in range(len(cluster)):
        for j in range(i):
            dist = metric(cluster[i], cluster[j])
            dists[i] += dist
            dists[j] += dist      
    min_dist = 10**100 # inf
    res = -1
    for i in range(len(dists)):
        if dists[i] < min_dist:
            min_dist = dists[i]
            res = i
    return cluster[res] 


def avg_dist(cluster, metric):
    if len(cluster) < 2:
        return 0
    if len(cluster) == 2:
        return metric(cluster[0], cluster[1])
    
    dist_sum = 0
    for i in range(len(cluster)):
        for j in range(i):
            dist_sum += metric(cluster[i], cluster[j])
    return dist_sum / ((len(cluster)-2) * (len(cluster)-1))

In [104]:
def davies_bouldin_index(clusters, metric, *args):
    centroids = [find_centroid(c, metric, *args) for c in clusters]
    avg_dists = [avg_dist(c, metric, *args) for c in clusters]
    n = len(clusters)
    tab = [max([(avg_dists[i] + avg_dists[j]) / metric(centroids[i], centroids[j], *args) \
                for i in range(n) if i != j]) for j in range(n)]
    return sum(tab) / n

### Ex 3

In [124]:
def create_stoplist(text, freq=0.5):
    """
    Creates a stoplist from a given text.
    """
    words = []
    for lien in text:
        words += lien.split()
    counter = Counter(words)
    stoplist = {key for key, value in counter.items() if value > freq*len(text)}

    result = []
    for line in text:
        result.append(' '.join([word for word in line.split() if word not in stoplist]))
    return result

### Ex 4

In [106]:
def read_text(file, n):
    with open(file, 'r', encoding='utf-8') as f:
        text = f.read().splitlines()
    return text[:n]

In [107]:
def save_clusters(file, clusters):
    with open(file, 'w', encoding='utf-8') as f:
        for c in clusters:
            for line in c:
                f.write(line+'\n')
            f.write('============\n\n')

In [108]:
def print_clusters(clusters, n=10):
    i = j =0
    while j < n and i < len(clusters):
        c = clusters[i]
        if len(c) != 1:    
            for line in c:
                print(line)
            print('============\n')
            j += 1
        i += 1

In [109]:
def perform_clustering(text, metric_f, eps, stop_list_frequency=None, *args):
    working_text = text
    if stop_list_frequency is not None:
        working_text = create_stoplist(text, stop_list_frequency)
        
    def metric(x, y):
        i, j = int(x[0]), int(y[0])
        return metric_f(working_text[i], working_text[j])
    
    X = np.arange(len(working_text)).reshape(-1, 1)
    clustering = DBSCAN(metric=metric, min_samples=1, eps=eps).fit_predict(X)
    #print(clustering)
    clusters = [[] for i in range(max(clustering) + 1)]
    for i in range(len(clustering)):
        clusters[clustering[i]].append(text[i])
    return clusters

### Ex 5

In [110]:
text = read_text('lines.txt', 100)

In [117]:
def perform_tests(to_test, stop_list, res_v, eps_v):
    if stop_list:
        frequency = 0.01
    else:
        frequency = None
    for i in range(len(to_test)):
        clusters = perform_clustering(text, to_test[i], eps_v[i], frequency)
        print("Funkcja: ",to_test[i].__name__)
        print("Epsilon: ",eps_v[i])
        print("Indeks DB: ", davies_bouldin_index(clusters, to_test[i]))
        print("========\n")
        if stop_list:
            save_clusters(to_test[i].__name__ + "_stop.txt", clusters)
        else:
            save_clusters(to_test[i].__name__ + "_no_stop.txt", clusters)
        res_v.append(clusters)

In [115]:
def print_all(res_v, to_test):
    for i in range(len(res_v)//2):
        print("Klasteryzacja przy pomocy " + to_test[i].__name__ + " bez stoplisty:\n\n")
        print_clusters(res_v[i])
        print("**********************************************************************\n\n\n")
    for i in range(len(res_v)//2 ,len(res_v)):
        print("Klasteryzacja przy pomocy " + to_test[i - len(res_v)//2].__name__ + " ze stoplistą:\n\n")
        print_clusters(res_v[i])
        print("**********************************************************************\n\n\n")

In [130]:
to_test = [lcs_metric, cosine_metric, euclidean_metric, dice_metric]
res_v = []
eps_v = [0.3, 0.3, 0.7, 0.45]
res_vec_stop = []
print("Test bez stoplisty\n\n")
perform_tests(to_test, False, res_v, eps_v)  
print("\nTest ze stoplistą usuwającą wyrazy częstsze niż 0.01 * liczba wyrazów\n\n")
perform_tests(to_test, True, res_v, eps_v)

Test bez stoplisty


Funkcja:  lcs_metric
Epsilon:  0.3
Indeks DB:  0.25904232112018055

Funkcja:  cosine_metric
Epsilon:  0.3
Indeks DB:  0.45212932041620785

Funkcja:  euclidean_metric
Epsilon:  0.7
Indeks DB:  0.0

Funkcja:  dice_metric
Epsilon:  0.45
Indeks DB:  0.6357253431441922


Test ze stoplistą usuwającą wyrazy częstsze niż 0.01 * liczba wyrazów


Funkcja:  lcs_metric
Epsilon:  0.3
Indeks DB:  0.6210573197606084

Funkcja:  cosine_metric
Epsilon:  0.3
Indeks DB:  0.8322517512962989

Funkcja:  euclidean_metric
Epsilon:  0.7
Indeks DB:  0.65856443133609

Funkcja:  dice_metric
Epsilon:  0.45
Indeks DB:  0.7303176004602366



### Ex 6

Można zmienić litery na taką samą wielkość

Można szukać lepszych parametrów na przykład do stworzenia stoplisty